# BO runs

In [2]:
!pip install botorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 4.9 MB/s eta 0:00:000:00:01m eta 0:00:01
  Using cached pyro_ppl-1.8.4-py3-none-any.whl (730 kB)
  Using cached multipledispatch-0.6.0-py3-none-any.whl (11 kB)
  Using cached gpytorch-1.9.1-py3-none-any.whl (250 kB)
  Using cached linear_operator-0.3.0-py3-none-any.whl (155 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.4 MB/s eta 0:00:004 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.12.0.dev20221110 requires astunparse>=1.6.0, which is not installed.
tf-nightly 2.12.0.dev20221110 requires gast<=0.4.0,>=0.2.1, which is not installed.
tf-nightly 2.12.0.dev20221110 requires google-pasta>=0.1.1, which is not installed.
tf-nightly 2.12.0.dev20221

In [1]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from scipy.stats import norm
from botorch.acquisition.analytic import ExpectedImprovement
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
import time

load data from `prepare_Xy.ipynb`

In [2]:
X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
y = np.reshape(y, (np.size(y), 1)) # for the GP
print("shape of y:", np.shape(y))

nb_COFs = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_COFs']
print("# COFs:", nb_COFs)

nb_iterations = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_iterations']
print("# iterations:", nb_iterations)

nb_runs = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_runs']
print("# runs:", nb_runs)

shape of X: (98694, 7)
shape of y: (98694, 1)
# COFs: 98694
# iterations: 100
# runs: 50


convert to torch tensors

In [3]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [4]:
X.size()

torch.Size([98694, 7])

In [5]:
y.size()

torch.Size([98694, 1])

In [6]:
X_unsqueezed = X.unsqueeze(1)

In [7]:
"""
BO run.

* nb_iterations: # of COFs to acquire = # of iterations in BO
* nb_COFs_initialization: # of COFs to acquire at random to initiate the GP and BO.
* which_acquisition: the acquisition function to use.
* store_explore_exploit_terms: True if you want to store (exploration contribution, exploitation contribution) to EI
"""
def bo_run(nb_iterations, nb_COFs_initialization, which_acquisition, verbose=False, store_explore_exploit_terms=False):
    assert nb_iterations > nb_COFs_initialization
    assert which_acquisition in ['max y_hat', 'EI', 'max sigma']
    
    # select initial COFs for training data randomly.
    # idea is to keep populating this ids_acquired and return it for analysis.
    ids_acquired = np.random.choice(np.arange((nb_COFs)), size=nb_COFs_initialization, replace=False)
    
    # keep track of exploration vs. exploitation terms ONLY for when using EI  
    if which_acquisition == "EI" and store_explore_exploit_terms:
        explore_exploit_balance = np.array([(np.NaN, np.NaN) for i in range(nb_iterations)])
    else:
        explore_exploit_balance = [] # don't bother

    # initialize acquired y, since it requires normalization
    y_acquired = y[ids_acquired]
    # standardize outputs using *only currently acquired data*
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i, end="\r")
        # construct and fit GP model
        model = SingleTaskGP(X[ids_acquired, :], y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # set up acquisition function
        if which_acquisition == "EI":
            acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
            
            # if having memory problems, compute EI this way, in batches :)
#             batch_size = 35000 # need to do in batches to avoid mem issues
#             acquisition_values = torch.zeros((nb_COFs))
#             acquisition_values[:] = np.NaN # for safety
#             nb_batches = nb_COFs // batch_size
#             for ba in range(nb_batches+1):
#                 id_start = ba * batch_size
#                 id_end   = id_start + batch_size
#                 if id_end > nb_COFs:
#                     id_end = nb_COFs
#                 with torch.no_grad():
#                     acquisition_values[id_start:id_end] = acquisition_function.forward(X_unsqueezed[id_start:id_end])
#             assert acquisition_values.isnan().sum().item() == 0 # so that all are filled properly.
            with torch.no_grad(): # to avoid memory issues; we arent using the gradient...
                acquisition_values = acquisition_function.forward(X_unsqueezed) # runs out of memory
        elif which_acquisition == "max y_hat":
            with torch.no_grad():
                acquisition_values = model.posterior(X_unsqueezed).mean.squeeze()
        elif which_acquisition == "max sigma":
            with torch.no_grad():
                acquisition_values = model.posterior(X_unsqueezed).variance.squeeze()
        else:
            raise Exception("not a valid acquisition function")

        # select COF to acquire with maximal aquisition value, which is not in the acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, [id_max_aquisition]))
        assert np.size(ids_acquired) == i + 1
        
        # if EI, compute and store explore-exploit terms that contribute to EI separately.
        if which_acquisition == "EI" and store_explore_exploit_terms:
            # explore, exploit terms of EI. requires computing EI manually, essentially. 
            y_pred = model.posterior(X_unsqueezed[id_max_aquisition]).mean.squeeze().detach().numpy()
            sigma_pred = np.sqrt(model.posterior(X_unsqueezed[id_max_aquisition]).variance.squeeze().detach().numpy())
            
            y_max = y_acquired.max().item()
            
            z = (y_pred - y_max) / sigma_pred
            explore_term = sigma_pred * norm.pdf(z)
            exploit_term = (y_pred - y_max) * norm.cdf(z)
            
            # check we computed it right... i.e. that it agrees with BO torch's EI.
            assert np.isclose(explore_term + exploit_term, acquisition_values[id_max_aquisition].item())

            explore_exploit_balance[i] = (explore_term, exploit_term)

        # update y aquired; start over to normalize properly
        y_acquired = y[ids_acquired] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
        
        if verbose:
            print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition].item())
            print("\tbest y acquired:", y[ids_acquired].max().item())
        
    assert np.size(ids_acquired) == nb_iterations
    return ids_acquired, explore_exploit_balance

must run with `which_acquisition` equal to all three below.

In [11]:
# which_acquisition = "EI"
# which_acquisition = "max y_hat"
which_acquisition = "max sigma"
nb_COFs_initializations = {"EI": [5, 10, 15, 20, 25], 
                           "max y_hat": [10], 
                           "max sigma": [10]}

for nb_COFs_initialization in nb_COFs_initializations[which_acquisition]:
    print("# COFs in initialization:", nb_COFs_initialization)
    # store results here.
    bo_res = dict() 
    bo_res['ids_acquired']            = []
    bo_res['explore_exploit_balance'] = []
    
    if nb_COFs_initialization == 10 and which_acquisition == 'EI':
        store_explore_exploit_terms = True
    else:
        store_explore_exploit_terms = False
    
    for r in range(nb_runs):
        print("\nRUN", r)
        t0 = time.time()
        
        ids_acquired, explore_exploit_balance = bo_run(nb_iterations, nb_COFs_initialization, which_acquisition, store_explore_exploit_terms=store_explore_exploit_terms)
        
        # store results from this run.
        bo_res['ids_acquired'].append(ids_acquired)
        bo_res['explore_exploit_balance'].append(explore_exploit_balance)
        
        print("took time t = ", (time.time() - t0) / 60, "min\n")
    
    # save results from all runs
    with open('bo_results_' + which_acquisition + "_initiate_with_{0}".format(nb_COFs_initialization) + '.pkl', 'wb') as file:
        pickle.dump(bo_res, file)
        
with open('bo_results_nb_COF_initializations.pkl', 'wb') as file:
    pickle.dump(nb_COFs_initializations, file)

# COFs in initialization: 10

RUN 0
took time t =  2.491740079720815 min


RUN 1
took time t =  2.3598769982655843 min


RUN 2
took time t =  2.2923808137575787 min


RUN 3
took time t =  2.4538424968719483 min


RUN 4
took time t =  2.2930290619532268 min


RUN 5
took time t =  2.452386287848155 min


RUN 6
took time t =  2.5364050308863324 min


RUN 7
took time t =  2.28095672527949 min


RUN 8
took time t =  2.3892273267110187 min


RUN 9
took time t =  2.3227869510650634 min


RUN 10
took time t =  2.3925676941871643 min


RUN 11
took time t =  2.3349881052970884 min


RUN 12
took time t =  2.339087728659312 min


RUN 13
took time t =  2.480728538831075 min


RUN 14
took time t =  2.342625153064728 min


RUN 15
took time t =  2.28629891872406 min


RUN 16
took time t =  2.565360959370931 min


RUN 17


/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/lin

took time t =  2.4627824306488035 min


RUN 18
took time t =  2.409364386399587 min


RUN 19


/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/lin

took time t =  2.6117578665415446 min


RUN 20
took time t =  2.315704945723216 min


RUN 21
took time t =  2.382988409201304 min


RUN 22
took time t =  2.2967713793118794 min


RUN 23
took time t =  2.311958742141724 min


RUN 24
took time t =  2.488028864065806 min


RUN 25
took time t =  2.3035463213920595 min


RUN 26
took time t =  2.3117849826812744 min


RUN 27
took time t =  2.366830825805664 min


RUN 28
took time t =  2.278993240992228 min


RUN 29
took time t =  2.1843421936035154 min


RUN 30
took time t =  2.409311560789744 min


RUN 31
took time t =  2.3542076071103413 min


RUN 32
took time t =  2.131477439403534 min


RUN 33
took time t =  2.244883648554484 min


RUN 34
took time t =  2.3656041304270428 min


RUN 35
took time t =  2.3882257024447124 min


RUN 36
took time t =  2.559925897916158 min


RUN 37
took time t =  2.4093440731366473 min


RUN 38
took time t =  2.414399512608846 min


RUN 39
took time t =  2.225052313009898 min


RUN 40
took time t =  2.20104854